In [13]:
import pandas as pd
import torch
import torchvision
import torch.nn as nn
from tqdm import tqdm

from datasets.robot_hand import RobotHandDataset

In [14]:
dataset = RobotHandDataset('test', '/mnt/d/data/robot_hand', mode='full')
data_loader = torch.utils.data.DataLoader(dataset, batch_size=1)

model = torchvision.models.resnet50()
model.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3,bias=False)
model.fc = nn.Linear(2048, 12)
model.load_state_dict(torch.load('logs/14.pth'))

<All keys matched successfully>

In [15]:
outfile = 'resnet50_epoch_14.csv'


output_file = open(outfile, 'w')

titles = ['ID', 'FINGER_POS_1', 'FINGER_POS_2', 'FINGER_POS_3', 'FINGER_POS_4', 'FINGER_POS_5', 'FINGER_POS_6',
         'FINGER_POS_7', 'FINGER_POS_8', 'FINGER_POS_9', 'FINGER_POS_10', 'FINGER_POS_11', 'FINGER_POS_12']
preds = []

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)
model.eval()

with torch.no_grad():
    for image_input, label in tqdm(data_loader):
        image_input = image_input.to(device)

        output = model(image_input.to(device)) / 1000
        preds.append(output[0].cpu().detach().numpy())

df = pd.concat([pd.DataFrame(dataset.field_ids), pd.DataFrame.from_records(preds)], axis = 1, names = titles)
df.columns = titles
df.to_csv(outfile, index = False)
print("Written to csv file {}".format(outfile))

100%|██████████| 849/849 [00:21<00:00, 39.03it/s]

Written to csv file resnet50_epoch_14.csv


In [16]:
len(dataset.field_ids)

849